<a href="https://colab.research.google.com/github/HoJeong-LEE/Seocho-2019/blob/master/Sentiment_Analysis_on_IMDb_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Trial


*   standard RNN for RNN layer
*   SGD for optimizer

**Preparing Data**

In [0]:
import torch

In [0]:
from torchtext import data

In [0]:
SEED = 1234

In [0]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [0]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 40.3MB/s]


In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [0]:
print(vars(train_data.examples[0]))

{'text': ['This', 'was', 'without', 'a', 'doubt', 'the', 'best', 'of', 'the', '"', 'Dirty', 'Harry', '"', 'series', '.', 'From', 'the', 'opening', 'credits', ',', 'you', "'re", 'swept', 'up', 'in', 'a', 'revenge', 'tale', 'that', 'hits', 'hard', 'and', 'is', 'profoundly', 'engrossing', '.', 'Sondra', 'Locke', 'is', 'perfect', 'in', 'the', 'role', 'of', 'a', 'traumatized', 'woman', 'out', 'for', 'revenge', '.', 'Eastwood', 'has', 'many', '"', 'aside', '"', 'sequences', 'that', 'have', 'nothing', 'to', 'do', 'with', 'the', 'plot', ',', 'but', 'show', 'Harry', 'at', 'his', 'bad', '-', 'assed', 'best', '.', 'Loaded', 'with', 'unforgettable', 'characters', 'in', 'minor', 'roles', ',', 'this', 'film', 'rocks', 'and', 'should', 'serve', 'as', 'the', 'standard', 'for', 'detective', '/', 'action', 'flicks', '.', 'This', 'is', 'the', 'one', 'Dirty', 'Harry', 'flick', 'that', "'s", 'raw', 'and', 'devoid', 'of', 'any', '"', 'fluff', '"', '.', 'I', 'can', 'watch', 'this', 'again', 'and', 'again', '

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
MAX_VOCAB_SIZE = 60000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 60002
Unique tokens in LABEL vocabulary: 2


In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203459), (',', 192975), ('.', 164951), ('and', 109566), ('a', 109524), ('of', 100784), ('to', 93643), ('is', 76277), ('in', 61706), ('I', 54550), ('it', 53702), ('that', 49563), ('"', 44324), ("'s", 43067), ('this', 42411), ('-', 37120), ('/><br', 35602), ('was', 35107), ('as', 30575), ('with', 29913)]


In [0]:
import torch
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

**Building the Model**

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)        
        output, hidden = self.rnn(embedded)
          
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,092,105 trainable parameters


**Training and Testing**

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01)

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 12s
	Train Loss: 0.694 | Train Acc: 49.86%
	 Val. Loss: 0.695 |  Val. Acc: 50.26%
Epoch: 02 | Epoch Time: 1m 11s
	Train Loss: 0.694 | Train Acc: 49.84%
	 Val. Loss: 0.695 |  Val. Acc: 50.06%
Epoch: 03 | Epoch Time: 1m 11s
	Train Loss: 0.694 | Train Acc: 49.84%
	 Val. Loss: 0.695 |  Val. Acc: 49.56%
Epoch: 04 | Epoch Time: 1m 11s
	Train Loss: 0.694 | Train Acc: 50.06%
	 Val. Loss: 0.697 |  Val. Acc: 48.45%
Epoch: 05 | Epoch Time: 1m 11s
	Train Loss: 0.694 | Train Acc: 49.86%
	 Val. Loss: 0.696 |  Val. Acc: 48.08%


In [0]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.687 | Test Acc: 57.24%


**Result :**

Accuracy of **57.24%**

# Second Trial
* LSTM for RNN layer
* Adam for optimizer
* Dropout (regularization) applied


**Preparing Data**

In [0]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

**Building the Model**

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        # LSTM was used this time
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        # regularization was applied
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedded = self.dropout(self.embedding(text))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        return self.fc(hidden)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [59]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [60]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [61]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-3.6247, -1.2173, -0.7283,  ..., -0.6435, -0.2026, -0.3940],
        [-0.6058, -0.2772,  1.4801,  ..., -0.1984,  0.5430,  1.5206],
        [-0.5121,  0.2059,  0.5721,  ..., -0.7374, -0.2360,  0.3210]])

In [62]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-3.6247, -1.2173, -0.7283,  ..., -0.6435, -0.2026, -0.3940],
        [-0.6058, -0.2772,  1.4801,  ..., -0.1984,  0.5430,  1.5206],
        [-0.5121,  0.2059,  0.5721,  ..., -0.7374, -0.2360,  0.3210]])


**Training and Testing**

In [0]:
import torch.optim as optim
# Adam was used this time
optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [69]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 51s
	Train Loss: 0.663 | Train Acc: 60.59%
	 Val. Loss: 1.056 |  Val. Acc: 49.42%
Epoch: 02 | Epoch Time: 1m 52s
	Train Loss: 0.675 | Train Acc: 58.45%
	 Val. Loss: 0.647 |  Val. Acc: 66.87%
Epoch: 03 | Epoch Time: 1m 51s
	Train Loss: 0.518 | Train Acc: 74.97%
	 Val. Loss: 0.408 |  Val. Acc: 81.70%
Epoch: 04 | Epoch Time: 1m 52s
	Train Loss: 0.386 | Train Acc: 83.27%
	 Val. Loss: 0.525 |  Val. Acc: 72.55%
Epoch: 05 | Epoch Time: 1m 51s
	Train Loss: 0.328 | Train Acc: 86.32%
	 Val. Loss: 0.346 |  Val. Acc: 85.52%


In [70]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.351 | Test Acc: 85.24%


**Result :**

Accuracy of **85.24%**

# Third Trial
embedding 방법 바꾸기 →Word2Vec

In [0]:
from gensim.models import Word2Vec

In [80]:
embedding_model = Word2Vec(TEXT.vocab, min_count=1)

TypeError: ignored